In [2]:
%reload_ext autoreload
%autoreload 2
import sys
sys.path.append('/home/sebastian/masters/') # add my repo to python path
import os
import torch
import torch_geometric
import kmbio  # fork of biopython PDB with some changes in how the structure, chain, etc. classes are defined.
import numpy as np
import proteinsolver
import modules
from modules.dataset import *

np.random.seed(1)

In [4]:
state_file = "/home/sebastian/data/e53-s1952148-d93703104.state"
root = "/home/sebastian/masters/data/210916_TCRpMHCmodels/"
save_path = "/home/sebastian/masters/data/trained_models/test_model.state"

### Load positive examples and split into structure configurations

In [23]:
class ChainFilter(kmbio.PDB.Select):
    def __init__(self, subset):
        self.subset = subset

    def accept_chain(self, chain):
        if chain.id in self.subset:
            return 1
        else:
            return 0

raw_files, targets = get_data(
    model_dir=f"{root}/models/",
    model_name=f"model_TCR-pMHC.pdb",
    metadata=f"{root}/train_data_gene_names.csv",
)
pos_raw_files = np.ma.masked_array(raw_files, mask=targets)  # only get positives

pmhc_chain_subset = ["M", "P"]
p_chain_subset = ["P"]

for raw_file in pos_raw_files:
    
    common_path = os.path.dirname(raw_file)
    pmhc_file_name = f"{common_path}/model_pMHC.pdb"
    p_file_name =  f"{common_path}/model_p.pdb"
    
    structure  = kmbio.PDB.load(raw_file)
    
    io = kmbio.PDB.io.PDBIO()
    io.set_structure(structure)
    io.save(pmhc_file_name, ChainFilter(subset=pmhc_chain_subset))
    
    io = kmbio.PDB.io.PDBIO()
    io.set_structure(structure)
    io.save(p_file_name, ChainFilter(subset=p_chain_subset))
    



#for each file:
#    load into bio pdb
#    make new structures for the two new configs (one config is just the normal, so 3 total)
#    save the new structures in some folder if not already present
#    add each 3 paths to 3 list (i.e. a list with files for each config)
#
#pass each list to normal data and make pre process


SyntaxError: invalid syntax (2078894121.py, line 9)

In [ ]:
raw_files, targets = get_data(
    model_dir=f"{root}/models/",
    model_name=f"model_TCR-pMHC.pdb",
    metadata=f"{root}/train_data_gene_names.csv",
)

targets = targets[targets]
pos_binders = raw_files[targets]






# make quick data split
n_data = len(raw_files)
valid_frac = 0.1
valid_num = int(n_data * valid_frac)
selection = np.random.randint(0, n_data, valid_num)
mask = np.zeros(n_data, bool)
mask[selection] = 1

valid_files = raw_files[mask]
valid_targets = targets[mask]
d_valid = ProteinDataset(f"{root}/valid", valid_files, valid_targets, overwrite=True)

train_files = raw_files[~mask]
train_targets = targets[~mask]
d_train = ProteinDataset(f"{root}/train", train_files, train_targets, overwrite=True)

### Generate datasets for different structure configurations